In [25]:
%pip install langchain-ollama pydantic langsmith

Note: you may need to restart the kernel to use updated packages.


In [26]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"]="pr-wilted-special-58"

In [27]:
from pydantic import BaseModel, Field

class Response(BaseModel):
    question: str = Field(description="The question asked.")
    answer: str = Field(description="The answer to the question.")
    reasoning: str = Field(description="The reasoning behind the answer.")

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser

system_prompt = """You are a helpful assistant. You will be given a question and you need to answer it step by step."""

template = """Question: {question}

Answer: Let's think step by step."""

parser = StrOutputParser()


prompt = ChatPromptTemplate([
  ("system", system_prompt),
  ("human", template),
])

model = ChatOllama(model="deepseek-r1:7b", base_url="http://localhost:11434")
# .with_structured_output(Response)

chain = prompt | model | parser

response = chain.invoke({"question": "What is the capital of Thailand?"})
response

'<think>\nOkay, so I need to figure out what the capital of Thailand is. Hmm, where do I start? I know that Thailand is one of those countries in Southeast Asia, right? It\'s pretty famous for stuff like beaches and tourist spots. But when it comes to capitals, I\'m a bit fuzzy on the details.\n\nI remember that many Asian countries have capitals somewhere else than their main city. For example, Japan has Tokyo as its capital instead of Kyoto or Hakone. So maybe Thailand isn\'t different? Wait, isn\'t Bangkok a big city in Thailand? I think it\'s also known as the "Goldenzent" because of all the yellow things there.\n\nWait, so is Bangkok the capital? That sounds right. But how can I be sure? Maybe I should think about other countries nearby. Like, what\'s the capital of Indonesia? It\'s Jakarta, which isn\'t its main city but a big port city instead. So maybe Thailand follows a similar pattern?\n\nI\'ve heard people talk about going to Bangkok for travel or business, so it must be the